In [3]:
import numpy as np
import tensorflow as tf

In [4]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
set_memory_growth()

1 Physical GPUs, 1 Logical GPUs


In [67]:
N = 10
xt, yt = np.arange(N), np.arange(10)

In [68]:
xt

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [69]:
ds = tf.data.Dataset.from_tensor_slices(xt)
list(ds.as_numpy_iterator())
len(ds)

10

In [70]:
from typing import Tuple

In [113]:
class DS(tf.keras.utils.Sequence):
    def __init__(self, name: str, n: int, b: int):
        self.name = name
        self.n = n
        self.b = b
        self.x = np.random.uniform(size=(self.n, 10, 10, 3))
        self.y = np.random.randint(0, 2, size=(self.n, 10))
    
    def __len__(self) -> int:
        return self.n // self.b + self.n % self.b

    def __getitem__(self, ind: int) -> Tuple[np.ndarray]:
        inds = slice(self.b * ind, self.b * (ind + 1))
        x, y = self.x[inds], self.y[inds]
        print(f'DS({self.name}).__getitem__', ind, len(x))
        return self.x[inds], self.y[inds]
    
    def on_epoch_end(self):
        print(f'DS({self.name}).on_epoch_end')
        
    def gen(self):
        nb = self.n // self.b + self.n % self.b
        ib = 0
        while True:
            inds = slice(self.b * ib, self.b * (ib + 1))
            x, y = self.x[inds], self.y[inds]
            print(f'DS({self.name})', ib, len(x))
            yield x, y
            ib += 1
            if ib == nb:
                print(f'DS({self.name}) is over')
                ib = 0


In [114]:
def get_uncompiled_model():
    inp = tf.keras.Input(shape=(10, 10, 3), dtype=tf.float32)
    x = inp
    x = tf.keras.layers.Conv2D(5, 3, activation='relu')(x)
    x = tf.keras.layers.Conv2D(10, 3, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Softmax()(x)
    out = x
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    return model


In [115]:
model = get_uncompiled_model()
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

In [116]:
ds_train = DS('train', 20, 4)
ds_val = DS('val', 10, 4)

In [104]:
model.fit(ds_train, epochs=3, steps_per_epoch=2, validation_data=ds_val, validation_steps=2,
          max_queue_size=0, workers=0, shuffle=True)

DS(train).__getitem__ 0 4
Epoch 1/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.2495 - categorical_accuracy: 0.0000e+00DS(train).__getitem__ 1 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 1 4
DS(val).on_epoch_end
2/2 [==============================] - 0s 63ms/step - loss: 12.3325 - categorical_accuracy: 0.1250 - val_loss: 11.9955 - val_categorical_accuracy: 0.1250
DS(train).on_epoch_end
Epoch 2/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.3490 - categorical_accuracy: 0.0000e+00DS(train).__getitem__ 1 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 1 4
DS(val).on_epoch_end
2/2 [==============================] - 0s 43ms/step - loss: 12.4135 - categorical_accuracy: 0.1250 - val_loss: 12.0610 - val_categorical_accuracy: 0.1250
DS(train).on_epoch_end
Epoch 3/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.4533 - categorical_accuracy: 0.0000e+00DS(trai

In [117]:
out_sig = (
        tf.TensorSpec(shape=(None, 10, 10, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32),
    )
ds_train_gen = tf.data.Dataset.from_generator(ds_train.gen, output_signature=out_sig)
ds_val_gen = tf.data.Dataset.from_generator(ds_val.gen, output_signature=out_sig)
model.fit(ds_train_gen, epochs=3, steps_per_epoch=2, validation_data=ds_val_gen, validation_steps=2,
          max_queue_size=0, workers=0, shuffle=True)

Epoch 1/3
DS(train) 0 4
1/2 [==============>...............] - ETA: 0s - loss: 12.1399 - categorical_accuracy: 0.2500DS(train) 1 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 149ms/step - loss: 11.1460 - categorical_accuracy: 0.1667 - val_loss: 10.9498 - val_categorical_accuracy: 0.0000e+00
Epoch 2/3
DS(train) 2 4
1/2 [==============>...............] - ETA: 0s - loss: 10.4354 - categorical_accuracy: 0.0000e+00DS(train) 3 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 34ms/step - loss: 10.8114 - categorical_accuracy: 0.0000e+00 - val_loss: 10.9561 - val_categorical_accuracy: 0.0000e+00
Epoch 3/3
DS(train) 4 4
1/2 [==============>...............] - ETA: 0s - loss: 11.4056 - categorical_accuracy: 0.2500DS(train) is over
DS(train) 0 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 38ms/step - loss: 11.6635 - categorical_accuracy: 0.2500 - val_loss: 10.9698 - val_categorical_accuracy: 0.0000e+00


In [20]:
np.random.uniform(0, 2, size=(2, 3)).astype(int)

array([[0, 0, 0],
       [0, 1, 1]])

In [31]:
n = 8
b = 4

In [32]:
n // b, n % b

(2, 0)

In [33]:
l = np.arange(7)

In [34]:
l

array([0, 1, 2, 3, 4, 5, 6])

In [35]:
l[2:3]

array([2])

In [40]:
l[slice(4, 7)]

array([4, 5, 6])

In [3]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy()
scce(y_true, y_pred).numpy()

1.1769392

In [4]:
yt = np.array(y_true)[..., None]
yp = np.array(y_pred)
print(yt)
print(yp)
print(scce(yt, yp))

[[1]
 [2]]
[[0.05 0.95 0.  ]
 [0.1  0.8  0.1 ]]
tf.Tensor(1.1769392490386963, shape=(), dtype=float64)


In [4]:
t = tf.random.uniform((2, 3), 0, 10, tf.int32)
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[5, 8, 6],
       [5, 4, 8]], dtype=int32)>

In [8]:
u, _ = tf.unique(tf.reshape(t, -1))
u

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([5, 8, 6, 4], dtype=int32)>

In [6]:
tf.reshape(t, -1)

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([5, 8, 6, 5, 4, 8], dtype=int32)>

In [13]:
for x in list(u.numpy()):
    print(x.item(), type(x))

5 <class 'numpy.int32'>
8 <class 'numpy.int32'>
6 <class 'numpy.int32'>
4 <class 'numpy.int32'>


In [17]:
for x in u:
    print(int(x), type(int(x)))

5 <class 'int'>
8 <class 'int'>
6 <class 'int'>
4 <class 'int'>


In [18]:
c = (1, 2)

In [19]:
t = tf.zeros((4, 2), np.uint8)
t

<tf.Tensor: shape=(4, 2), dtype=uint8, numpy=
array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=uint8)>

In [21]:
t[:2] += c

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [24]:
a = np.random.randint(0, 10, (4, 2), np.uint8)
a

array([[5, 4],
       [6, 5],
       [4, 3],
       [9, 8]], dtype=uint8)

In [26]:
u = np.unique(a)
u

array([3, 4, 5, 6, 8, 9], dtype=uint8)

In [31]:
b = np.random.random((4, 2, 2))
b

array([[[0.31200006, 0.67915729],
        [0.86853673, 0.83548093]],

       [[0.73041142, 0.64865287],
        [0.58436323, 0.61314648]],

       [[0.8027759 , 0.31762786],
        [0.62465495, 0.9026469 ]],

       [[0.40696541, 0.30224006],
        [0.3530455 , 0.01182745]]])

In [32]:
b[a == 4] = (-1, -2)

In [33]:
b

array([[[ 0.31200006,  0.67915729],
        [-1.        , -2.        ]],

       [[ 0.73041142,  0.64865287],
        [ 0.58436323,  0.61314648]],

       [[-1.        , -2.        ],
        [ 0.62465495,  0.9026469 ]],

       [[ 0.40696541,  0.30224006],
        [ 0.3530455 ,  0.01182745]]])

In [35]:
tf.convert_to_tensor(u)

<tf.Tensor: shape=(6,), dtype=uint8, numpy=array([3, 4, 5, 6, 8, 9], dtype=uint8)>

In [37]:
tf.convert_to_tensor(b)

<tf.Tensor: shape=(4, 2, 2), dtype=float64, numpy=
array([[[ 0.31200006,  0.67915729],
        [-1.        , -2.        ]],

       [[ 0.73041142,  0.64865287],
        [ 0.58436323,  0.61314648]],

       [[-1.        , -2.        ],
        [ 0.62465495,  0.9026469 ]],

       [[ 0.40696541,  0.30224006],
        [ 0.3530455 ,  0.01182745]]])>

In [9]:
a = tf.random.uniform((2, 3), 0, 10, dtype=tf.int32)
a

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[9, 9, 0],
       [6, 2, 0]], dtype=int32)>

In [12]:
af = tf.cast(a, tf.float32)
tf.linalg.normalize(af, axis=0)

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0.83205026, 0.97618705,        nan],
        [0.5547002 , 0.21693046,        nan]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[10.816654,  9.219544,  0.      ]], dtype=float32)>)

In [25]:
a = tf.random.uniform((2, 3, 2))
a

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[0.47196913, 0.8774214 ],
        [0.1535958 , 0.77537155],
        [0.66445637, 0.07393742]],

       [[0.18289328, 0.87432253],
        [0.12357914, 0.28364694],
        [0.01825964, 0.25285006]]], dtype=float32)>

In [39]:
x = a.numpy()
xn = np.linalg.norm(x, axis=-1)
xm = xn >= 0.3
x.shape, xn.shape, xm.shape

((2, 3, 2), (2, 3), (2, 3))

In [40]:
x[xm] /= xn[xm][..., None]
x

array([[[0.47371966, 0.88067573],
        [0.19431725, 0.9809387 ],
        [0.99386585, 0.11059247]],

       [[0.20475112, 0.9788141 ],
        [0.39941746, 0.91676915],
        [0.01825964, 0.25285006]]], dtype=float32)

In [34]:
xm

array([[ True,  True,  True],
       [ True,  True, False]])

In [28]:
xn

array([[0.9963047 , 0.79043835, 0.6685574 ],
       [0.89324677, 0.30939844, 0.2535085 ]], dtype=float32)

In [41]:
a

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[0.47196913, 0.8774214 ],
        [0.1535958 , 0.77537155],
        [0.66445637, 0.07393742]],

       [[0.18289328, 0.87432253],
        [0.12357914, 0.28364694],
        [0.01825964, 0.25285006]]], dtype=float32)>

In [42]:
tf.cast(a, tf.bool)

<tf.Tensor: shape=(2, 3, 2), dtype=bool, numpy=
array([[[ True,  True],
        [ True,  True],
        [ True,  True]],

       [[ True,  True],
        [ True,  True],
        [ True,  True]]])>

In [45]:
x = tf.random.uniform((2, 3), 0, 2, tf.int32)
x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 1, 0],
       [0, 1, 1]], dtype=int32)>

In [47]:
tf.cast(x, tf.bool)

<tf.Tensor: shape=(2, 3), dtype=bool, numpy=
array([[ True,  True, False],
       [False,  True,  True]])>

In [48]:
y = tf.cast(x, tf.float32)
tf.cast(y, tf.bool)

<tf.Tensor: shape=(2, 3), dtype=bool, numpy=
array([[ True,  True, False],
       [False,  True,  True]])>

In [49]:
len(x)

2

In [50]:
x.shape

TensorShape([2, 3])

In [51]:
l = list(range(10))
print(l)
np.random.shuffle(l)
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[3, 7, 4, 8, 5, 0, 2, 9, 6, 1]


In [56]:
x = tf.random.uniform((5, 2), 0, 1)
y = tf.random.uniform((5, 2), 0, 1)
print(x, y)

tf.Tensor(
[[0.06365311 0.63394547]
 [0.1756128  0.5204389 ]
 [0.90855193 0.7148839 ]
 [0.75107276 0.5391841 ]
 [0.67091084 0.08267534]], shape=(5, 2), dtype=float32) tf.Tensor(
[[0.64913106 0.38104343]
 [0.7203268  0.90687   ]
 [0.07305932 0.04641211]
 [0.099563   0.11231327]
 [0.65197647 0.6700978 ]], shape=(5, 2), dtype=float32)


In [57]:
x * y

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.04131921, 0.24156076],
       [0.12649861, 0.47197044],
       [0.06637818, 0.03317927],
       [0.07477906, 0.06055753],
       [0.43741807, 0.05540057]], dtype=float32)>

In [55]:
x[0, 0] * y[0, 0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.080847286>

In [66]:
n = tf.norm(x, axis=-1)
n

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.63713306, 0.5492691 , 1.156082  , 0.9245701 , 0.67598563],
      dtype=float32)>

In [68]:
x1 = tf.reduce_sum(x, axis=-1)
x1

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.6975986, 0.6960517, 1.6234359, 1.2902569, 0.7535862],
      dtype=float32)>

In [69]:
x1 / n

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([1.0949025, 1.2672327, 1.4042566, 1.3955209, 1.1147962],
      dtype=float32)>

In [87]:
a = np.random.uniform(0, 1, (2,2,3))
a

array([[[0.38783133, 0.2115935 , 0.87307943],
        [0.42604806, 0.11948292, 0.14302413]],

       [[0.15632606, 0.00918984, 0.24872676],
        [0.25252541, 0.34400804, 0.11861873]]])

In [88]:
b = np.linalg.norm(a, axis=-1)
b

array([[0.97849509, 0.46502583],
       [0.29391716, 0.44292326]])

In [93]:
bm = b > 0.4
bm

array([[ True,  True],
       [False,  True]])

In [97]:
a[bm], b[bm][..., None]

(array([[0.38783133, 0.2115935 , 0.87307943],
        [0.42604806, 0.11948292, 0.14302413],
        [0.25252541, 0.34400804, 0.11861873]]),
 array([[0.97849509],
        [0.46502583],
        [0.44292326]]))

In [98]:
a[bm] /= -b[bm][..., None]
a

array([[[-0.39635491, -0.2162438 , -0.89226756],
        [-0.9161815 , -0.25693825, -0.30756169]],

       [[ 0.15632606,  0.00918984,  0.24872676],
        [-0.57013355, -0.77667638, -0.26780875]]])

In [99]:
np.linalg.norm(a, axis=-1)

array([[1.        , 1.        ],
       [0.29391716, 1.        ]])

In [78]:
tf.convert_to_tensor(a)

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[ 0.35354987,  0.88606198,  0.06251439],
       [ 0.77455812,  0.34072981,  0.06961438],
       [-0.01133427, -0.69231146, -0.80417438],
       [-0.9208761 , -0.30762072, -0.11335117]])>

In [86]:
a.copy()

array([[0.65276334, 0.61901419, 0.16618695],
       [0.00716774, 0.0128801 , 0.81666247],
       [0.82226359, 0.05942693, 0.56599561],
       [0.21484257, 0.24601617, 0.7193696 ]])